<a href="https://colab.research.google.com/github/Ddkaba/IAD_Lab_2/blob/main/IAD_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Группа 4232

Спицов А.

Михайлов Д.

Вариант №2 (Анализ вин по трем производителям)

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, classification_report
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.impute import SimpleImputer
from sklearn.utils.class_weight import compute_class_weight
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [2]:
dataset = pd.read_csv("https://raw.githubusercontent.com/Ddkaba/IAD_Lab_2/main/V2_classification_lr3.csv", index_col=0)
if 'No' in dataset.columns:
    dataset = dataset.drop(columns=['No'])

In [ ]:
TARGET = 'target'

print("Общая информация")
print(dataset.info())

print(f"Количество записей (объектов): {dataset.shape[0]}")
print(f"Количество признаков (фич): {dataset.shape[1]}")

print("\nНазвания столбцов:")
print(dataset.columns.tolist())

print("\nТипы данных:")
print(dataset.dtypes)

print("\nПропущенные значения:")
missing_values = dataset.isnull().sum()
print(missing_values)
print(f"Общее количество пропущенных значений: {missing_values.sum()}")

print("Целевая переменная")
if TARGET in dataset.columns:
    print(f"\nЦелевая переменная: {TARGET}")
    print(f"Тип данных целевой переменной: {dataset[TARGET].dtype}")
    unique_values = dataset[TARGET].unique()
    print(f"Уникальные значения целевой переменной (первые 20): {unique_values[:20]}")
    print(f"Всего уникальных значений: {unique_values.size}")
    if dataset[TARGET].nunique() <= 20:
        print("Распределение классов:")
        print(dataset[TARGET].value_counts())
        print("Процентное соотношение классов:")
        print(dataset[TARGET].value_counts(normalize=True) * 100)

print("Статистика")
print(dataset.describe())

print("Анализ кат. признаков")
categorical_features = []
for col in dataset.columns:
    unique_values = dataset[col].nunique(dropna=True)
    if unique_values <= 10:
        categorical_features.append(col)
        print(f"{col}: {unique_values} уникальных значений - {dataset[col].unique()}")

print(f"\nВсего категориальных признаков: {len(categorical_features)}")

Общая информация
<class 'pandas.core.frame.DataFrame'>
Index: 178 entries, 14.23 to 14.13
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   malic_acid                    178 non-null    float64
 1   ash                           178 non-null    float64
 2   alcalinity_of_ash             178 non-null    float64
 3   magnesium                     178 non-null    float64
 4   total_phenols                 178 non-null    float64
 5   flavanoids                    178 non-null    float64
 6   nonflavanoid_phenols          178 non-null    float64
 7   proanthocyanins               178 non-null    float64
 8   color_intensity               178 non-null    float64
 9   hue                           178 non-null    float64
 10  od280/od315_of_diluted_wines  178 non-null    float64
 11  proline                       178 non-null    float64
 12  target                        178 non-null    

In [ ]:
# Корреляционная матрица для Wine датасета
correlation_matrix = dataset.corr(numeric_only=True)
plt.figure(figsize=(12, 10))

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Корреляционная матрица признаков Wine')
plt.tight_layout()
plt.show()

In [ ]:
# Гистограммы по всем числовым признакам
# Определим список числовых колонок и исключим целевую, если она есть
feature_columns = dataset.select_dtypes(include=[np.number]).columns.tolist()
if 'target' in feature_columns:
    feature_columns.remove('target')

_ = dataset[feature_columns].hist(
    bins=10,
    figsize=(20, 15),
    grid=False,
    edgecolor='black'
)
plt.suptitle('Распределение числовых признаков', y=1.02)
plt.tight_layout()
plt.show()


In [ ]:
df_original = dataset.copy()

In [ ]:
# Использование SelectKBest для оценки информативности признаков и выбора топ-5 (Wine)
source_df = dataset

# Числовые признаки
X_num = source_df.drop(columns=[TARGET]).select_dtypes(include=[np.number])
y = source_df[TARGET]

# Импутация оставшихся пропусков медианой (на всякий случай)
imputer = SimpleImputer(strategy='median')
X_num_imp = pd.DataFrame(imputer.fit_transform(X_num), columns=X_num.columns, index=X_num.index)

# Для классификации используем f_classif
all_selector = SelectKBest(score_func=f_classif, k='all')
all_selector.fit(X_num_imp, y)

# Результаты
scores_df = (
    pd.DataFrame({'feature': X_num_imp.columns, 'score': all_selector.scores_})
      .sort_values('score', ascending=False)
      .reset_index(drop=True)
)

print('Оценки информативности (f_classif), по убыванию:')
print(scores_df)

plt.figure(figsize=(10, 6))
sns.barplot(data=scores_df, x='score', y='feature', color='#1f77b4')
plt.title('SelectKBest: f_classif scores (Wine)')
plt.tight_layout()
plt.show()

# Выбор ТОП-K признаков
K = 5
selector = SelectKBest(score_func=f_classif, k=K)
selector.fit(X_num_imp, y)
selected_features = X_num_imp.columns[selector.get_support()].tolist()
print(f'Топ-{K} признаков:')
print(selected_features)


In [ ]:
# Инженерия признаков для Wine: доля флавоноидов в общих фенолах
EPS = 1e-9
dataset['flavonoid_share'] = dataset['flavanoids'] / (dataset['total_phenols'] + EPS)

# Проверка наличия целевой переменной в корреляционной матрице и вывод топ-корреляций
corr = dataset.corr()
if TARGET in corr.columns:
    print('\nТоп корреляций с целевой переменной:')
    print(corr[TARGET].sort_values(ascending=False).head(15))

In [ ]:
# Удаление нерелевантных признаков: ash и magnesium, затем корреляционная матрица
cols_to_drop = ['ash', 'magnesium']
actual_drop = [c for c in cols_to_drop if c in dataset.columns]
print('Удаляем признаки:', actual_drop)

dataset = dataset.drop(columns=actual_drop)
print('После удаления:', dataset.shape)
print('Текущие столбцы:')
print(dataset.columns.tolist())

df_engineered = dataset.copy()

# Корреляционная матрица после удаления
correlation_matrix = dataset.corr(numeric_only=True)
plt.figure(figsize=(12, 10))

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Корреляционная матрица после удаления (Wine)')
plt.tight_layout()
plt.show()

In [ ]:
# Стандартизация числовых признаков (z = (x - mean) / std) для Wine

# Базовые датафреймы
base_df = df_original
engineered_df = df_engineered

def standardize_features(dataset, target):
    """
    Стандартизирует числовые признаки (кроме целевой) и возвращает датафрейм с целевой.
    """
    if target not in dataset.columns:
        raise ValueError(f"Целевая переменная '{target}' отсутствует в датасете.")

    local_df = dataset.copy()
    feature_cols = local_df.select_dtypes(include=[np.number]).columns.tolist()
    if target in feature_cols:
        feature_cols.remove(target)
    if not feature_cols:
        raise ValueError("В датасете нет числовых признаков для стандартизации.")

    scaler = StandardScaler()
    scaled = scaler.fit_transform(local_df[feature_cols])
    standardized_df = pd.DataFrame(scaled, columns=feature_cols, index=local_df.index)

    df_preprocessed = standardized_df.join(local_df[[target]])

    print('Стандартизированы признаки:', feature_cols)
    print('Форма:', standardized_df.shape)
    means = standardized_df.mean().round(4)
    stdevs = standardized_df.std(ddof=0).round(4)
    print('\nСредние по столбцам (ожид. ≈ 0):')
    print(means)
    print('\nСт. отклонения (ожид. ≈ 1):')
    print(stdevs)
    print('\n')

    return df_preprocessed

df_original_preprocessed = standardize_features(base_df, TARGET)
df_engineered_preprocessed = standardize_features(engineered_df, TARGET)


In [ ]:
# Train/Validation/Test split: 60% / 20% / 20%

# Источник данных: используем расширенный набор
src = dataset if 'dataset' in globals() else dataset

X = src.drop(columns=[TARGET])
y = src[TARGET]

# 1) Test split (20%)
seed = 42
test_size = 0.2
val_size = 0.25  # 25% от train -> итог 60/20/20

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=seed
)

# 2) Validation split из обучающей части
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=val_size, random_state=seed
)

print('Shapes:')
print('X_train:', X_train.shape, 'X_val:', X_val.shape, 'X_test:', X_test.shape)
print('y_train:', y_train.shape, 'y_val:', y_val.shape, 'y_test:', y_test.shape)


In [ ]:
print("Общая информация")
print(df_original.info())
print('\n')
print(df_engineered.info())
print('\n')
print(df_original_preprocessed.info())
print('\n')
print(df_engineered_preprocessed.info())

In [ ]:
# Сравнение MLP-классификатора и SimpleRNN-классификатора на train/val для наборов:
# df_original, df_engineered_preprocessed, df_original_preprocessed, df_engineered
keras = tf.keras

# Фиксируем seed для воспроизводимости эксперимента
np.random.seed(42)
try:
    tf.random.set_seed(42)
except Exception:
    pass


# Проверяем, что данные уже разделены на train/val (иначе прерываем выполнение)
assert 'X_train' in globals() and 'X_val' in globals(), 'Сначала запустите ячейку с train/val/test split.'
train_idx = X_train.index
val_idx = X_val.index

# Составляем словарь доступных датасетов для тестирования
datasets_map = {}
if 'df_original' in globals():
    datasets_map['df_original'] = df_original.copy()
if 'df_engineered_preprocessed' in globals():
    datasets_map['df_engineered_preprocessed'] = df_engineered_preprocessed.copy()
if 'df_original_preprocessed' in globals():
    datasets_map['df_original_preprocessed'] = df_original_preprocessed.copy()
if 'df_engineered' in globals():
    datasets_map['df_engineered'] = df_engineered.copy()

assert len(datasets_map) > 0, 'Нет доступных датафреймов из списка.'

results = []

num_classes = int(dataset[TARGET].nunique())
reg = tf.keras.regularizers.l2(1e-4)

# Проходим по каждому датасету
for name, df in datasets_map.items():
    # Берем только числовые признаки
    X_all = df.drop(columns=[TARGET]).select_dtypes(include=[np.number])
    y_all = df[TARGET].astype(int)

    # Разделение на train/val по заранее сохранённым индексам
    X_tr = X_all.loc[train_idx]
    y_tr = y_all.loc[train_idx]
    X_va = X_all.loc[val_idx]
    y_va = y_all.loc[val_idx]

    # Class weights (на train)
    classes = np.unique(y_tr)
    cw = compute_class_weight(class_weight='balanced', classes=classes, y=y_tr)
    class_weight = {int(c): float(w) for c, w in zip(classes, cw)}

    # Препроцессинг: не стандартизируем повторно *_preprocessed
    is_preprocessed = name.endswith('_preprocessed')
    if is_preprocessed:
        X_tr_std = X_tr.values
        X_va_std = X_va.values
    else:
        imputer = SimpleImputer(strategy='median')
        scaler = StandardScaler()
        X_tr_imp = imputer.fit_transform(X_tr)
        X_va_imp = imputer.transform(X_va)
        X_tr_std = scaler.fit_transform(X_tr_imp)
        X_va_std = scaler.transform(X_va_imp)

    input_dim = X_tr_std.shape[1]

    # Callbacks
    es = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)

    # Полносвязная классификационная модель (MLP) с Dropout+L2
    mlp = keras.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(64, activation='relu', kernel_regularizer=reg),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu', kernel_regularizer=reg),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    mlp.compile(optimizer=keras.optimizers.Adam(1e-3), loss='sparse_categorical_crossentropy')
    mlp.fit(X_tr_std, y_tr.values, validation_data=(X_va_std, y_va.values),
            epochs=200, batch_size=32, verbose=0, callbacks=[es, rlrop], class_weight=class_weight)

    # Предсказания (классы) для MLP
    y_tr_prob = mlp.predict(X_tr_std, verbose=0)
    y_va_prob = mlp.predict(X_va_std, verbose=0)
    y_tr_pred = y_tr_prob.argmax(axis=1)
    y_va_pred = y_va_prob.argmax(axis=1)

    # Метрики MLP
    acc_tr = accuracy_score(y_tr, y_tr_pred)
    acc_va = accuracy_score(y_va, y_va_pred)
    bacc_tr = balanced_accuracy_score(y_tr, y_tr_pred)
    bacc_va = balanced_accuracy_score(y_va, y_va_pred)
    f1_micro_tr = f1_score(y_tr, y_tr_pred, average='micro')
    f1_micro_va = f1_score(y_va, y_va_pred, average='micro')
    f1_macro_tr = f1_score(y_tr, y_tr_pred, average='macro')
    f1_macro_va = f1_score(y_va, y_va_pred, average='macro')
    f1_weighted_tr = f1_score(y_tr, y_tr_pred, average='weighted')
    f1_weighted_va = f1_score(y_va, y_va_pred, average='weighted')

    results.append({
        'dataset': name,
        'model': 'MLP-Classifier',
        'acc_train': acc_tr,
        'acc_val': acc_va,
        'bal_acc_train': bacc_tr,
        'bal_acc_val': bacc_va,
        'f1_micro_train': f1_micro_tr,
        'f1_micro_val': f1_micro_va,
        'f1_macro_train': f1_macro_tr,
        'f1_macro_val': f1_macro_va,
        'f1_weighted_train': f1_weighted_tr,
        'f1_weighted_val': f1_weighted_va,
    })

    print(f"\n=== Классификация (MLP): {name} ===")
    print('Validation classification report:')
    print(classification_report(y_va, y_va_pred, digits=4))

    # SimpleRNN классификатор с dropout
    X_tr_seq = X_tr_std.reshape((-1, input_dim, 1))
    X_va_seq = X_va_std.reshape((-1, input_dim, 1))

    rnn = keras.Sequential([
        layers.Input(shape=(input_dim, 1)),
        layers.SimpleRNN(32, activation='tanh', dropout=0.2, recurrent_dropout=0.2),
        layers.Dense(num_classes, activation='softmax')
    ])
    rnn.compile(optimizer=keras.optimizers.Adam(1e-3), loss='sparse_categorical_crossentropy')
    rnn.fit(X_tr_seq, y_tr.values, validation_data=(X_va_seq, y_va.values),
            epochs=200, batch_size=32, verbose=0, callbacks=[es, rlrop], class_weight=class_weight)

    # Предсказания (классы) для RNN
    y_tr_prob_rnn = rnn.predict(X_tr_seq, verbose=0)
    y_va_prob_rnn = rnn.predict(X_va_seq, verbose=0)
    y_tr_pred_rnn = y_tr_prob_rnn.argmax(axis=1)
    y_va_pred_rnn = y_va_prob_rnn.argmax(axis=1)

    # Метрики RNN
    acc_tr_rnn = accuracy_score(y_tr, y_tr_pred_rnn)
    acc_va_rnn = accuracy_score(y_va, y_va_pred_rnn)
    bacc_tr_rnn = balanced_accuracy_score(y_tr, y_tr_pred_rnn)
    bacc_va_rnn = balanced_accuracy_score(y_va, y_va_pred_rnn)
    f1_micro_tr_rnn = f1_score(y_tr, y_tr_pred_rnn, average='micro')
    f1_micro_va_rnn = f1_score(y_va, y_va_pred_rnn, average='micro')
    f1_macro_tr_rnn = f1_score(y_tr, y_tr_pred_rnn, average='macro')
    f1_macro_va_rnn = f1_score(y_va, y_va_pred_rnn, average='macro')
    f1_weighted_tr_rnn = f1_score(y_tr, y_tr_pred_rnn, average='weighted')
    f1_weighted_va_rnn = f1_score(y_va, y_va_pred_rnn, average='weighted')

    results.append({
        'dataset': name,
        'model': 'SimpleRNN-Classifier',
        'acc_train': acc_tr_rnn,
        'acc_val': acc_va_rnn,
        'bal_acc_train': bacc_tr_rnn,
        'bal_acc_val': bacc_va_rnn,
        'f1_micro_train': f1_micro_tr_rnn,
        'f1_micro_val': f1_micro_va_rnn,
        'f1_macro_train': f1_macro_tr_rnn,
        'f1_macro_val': f1_macro_va_rnn,
        'f1_weighted_train': f1_weighted_tr_rnn,
        'f1_weighted_val': f1_weighted_va_rnn,
    })

    print(f"\n=== Классификация (SimpleRNN): {name} ===")
    print('Validation classification report:')
    print(classification_report(y_va, y_va_pred_rnn, digits=4))

# Итоговая таблица результатов
res_df = pd.DataFrame(results)
print('\nСводная таблица по метрикам (train/val):')
print(res_df.sort_values(['dataset', 'model']).to_string(index=False))